In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
primary_data=pd.read_excel(r"D:\bao.guanyu\LocGit\2018222\vehicle_parameter.xlsx")
primary_condition=pd.read_excel(r"D:\bao.guanyu\LocGit\2018222\c-wtvc.xlsx")
condition=primary_condition.iloc[:,1]
cwtvc_city=condition[0:900]
cwtvc_road=condition[900:1368]
cwtvc_highway=condition[1368:]

In [3]:
def viehicle_power(m_full,high,width,r,Cd,eff_trans,g,lamata,motor_e,motor_z,i1,i0):
    k=re.compile('\d+')
    motor_e=k.findall(motor_e)
    Pe=float(motor_e[0])
    Te=float(motor_e[2])
    motor_z=k.findall(motor_z)
    Pz=float(motor_z[0])
    nz=float(motor_z[1])
    Tz=float(motor_z[2])
    h=re.compile('\d+\.\d+')
    try:
        i1=h.findall(i1)
        i1=np.array(list(map(float,i1)))
    except:
        i1=float(i1)
        i1=np.array([i1,i1])
    A=width/1000*high/1000*0.81
    r=r/1000
    i0=float(i0)
    i1=i1*i0
    def vehicle_res(v,grade):
        f=0.0076+0.000056*v
        Ff=f*m_full*g*math.cos(math.atan(grade))
        Fw=Cd*A*v**2/21.15
        Fj=m_full*g*math.sin(math.atan(grade))
        Fres=Ff+Fw+Fj
        return Fres
    def vehicle_F(v,rate,peak):
        v=np.array(v)
        rate=np.array(rate)
        if peak=='peak':
            P=Pz
            T=Tz
        else:
            P=Pe
            T=Te
        n=v*rate/0.377/r
        T1=P*9550/n*eff_trans
        T1=np.array(T1)
        T1[T1>T]=T
        F=T1*rate/r
        return F
    
    #车速坡道车速爬坡度计算
    count=0
    grade_z=0
    v_max=np.zeros(len(i1))
    v_maxz=np.zeros(len(i1))
    v_4=np.zeros(len(i1))
    v_15=np.zeros(len(i1))
    for i_current in i1:
        v=np.arange(0,200,0.1)
        Fi1=vehicle_F(v,i_current,'rated')-vehicle_res(v,0)
        for i in range(1,len(v)):
            if Fi1[i]<0 or v[i]*i_current/0.377/r>nz:
                v_max[count]=v[i-1]
                break
            
        Fi1=vehicle_F(v,i_current,'peak')-vehicle_res(v,0)
        for i in range(1,len(v)):
            if Fi1[i]<0 or v[i]*i_current/0.377/r>nz:
                v_maxz[count]=v[i-1]
                break
        
        v=np.arange(0.1,200,0.1)
        Fi2=vehicle_F(v,i_current,'rated')-vehicle_res(v,0.04)
        for i in range(1,len(v)):
            if Fi2[i]<0 or v[i]*i_current/0.377/r>nz:
                v_4[count]=v[i-1]
                break
        
        v=np.arange(0,200,0.1) 
        Fi3=vehicle_F(v,i_current,'peak')-vehicle_res(v,0.15)
        for i in range(1,len(v)):
            if Fi3[i]<0 or v[i]*i_current/0.377/r>nz:
                v_15[count]=v[i-1]
                break
        count=count+1
    for grade in np.arange(1,0.04,-0.001):
        Fi4=vehicle_F(5,i1.max(),'peak')-vehicle_res(5,grade)
        if Fi4>0:
            grade_z=grade
            break
    
            
    '''
    print(Pe,Te,Pz,nz,Tz,i1,i0)
    print('持续最高车速为:%skm/h'%(v_max.max()))
    print('最大爬坡度:%s%%'%(grade_z*100))
    print('4%%坡道车速为:%skm/h'%(v_4.max()))
    print('15%%坡道车速为:%skm/h'%(v_15.max()))
    '''
    
    #加速时间计算   
    error_count=0
    v=0
    t=0
    dt=0.1
    flag30=1
    flag50=1
    flag80=1
    Fi5=np.zeros(len(i1))
    t_30=np.nan
    t_50=np.nan
    t_80=np.nan
    while v<100 :
        for i in range(0,len(i1)):
            Fi5[i]=vehicle_F(v,i1[i],'peak')-vehicle_res(v,0)
        a=Fi5.max()/lamata/m_full
        v=v+a*dt*3.6
        t=t+dt
        error_count=error_count+1
        if v>30 and flag30==1:
            t_30=t
            flag30=0
        if v>50 and flag50==1:
            t_50=t
            flag50=0
        if v>80 and flag80==1:
            t_80=t
            flag80=0
        if error_count>10000:
            break
    t_30=round(t_30,2)
    t_50=round(t_50,2)
    t_80=round(t_80,2)
    #print('0-30km/h加速时间:%s\n0-50km/h加速时间:%s\n0-80km/h加速时间:%s'%(t_30,t_50,t_80))
    Twz=Tz*i1.max()
    
    #能耗计算
    def W_equalspeed(v):
        P=vehicle_res(v,0)*v/3.6/1000
        W100=P/v*100
        return W100
    def W_cwtvc(cwtvc):
        W=0
        s=0
        for i in cwtvc:
            W=W+vehicle_res(i,0)*i/3.6/1000*1/3600
            s=s+i*1/3600
        W100=W/s*100
        return W100
    '''
    print('40km/h等速白公里能耗%s','60km/h等速白公里能耗%s',\
          'c-wtvc城市百公里能耗%s','c-wtvc公路百公里能耗%s','c-wtvc高速百公里能耗%s'\
          %(W_equalspeed(40),W_equalspeed(60),\
            W_cwtvc(cwtvc_city),W_cwtvc(cwtvc_road),W_cwtvc(cwtvc_highway)))
    '''
    
    return Twz,grade_z,v_max.max(),v_maxz.max(),v_4.max(),v_15.max(),t_30,t_50,t_80,\
W_equalspeed(40),W_equalspeed(60),W_cwtvc(cwtvc_city),W_cwtvc(cwtvc_road),W_cwtvc(cwtvc_highway)

In [4]:
data=primary_data.iloc[0:,1:]
data.index=primary_data.iloc[0:,0]
result=index_title=['轮端最大扭矩','最大爬坡','持续车速','最高车速','4%坡道车速','15%坡道车速',\
                    '0-30km/h','0-50km/h','0-80km/h',\
                    '40km/h等速百公里能耗','60km/h等速百公里能耗',\
                    'c-wtvc城市百公里能耗','c-wtvc公路百公里能耗','c-wtvc高速百公里能耗']
for i in data.columns:
    try:
        [m_full,high,width,r,Cd,eff_trans,lamata,g,motor_e,motor_z,i1,i0]=data[i][0:12]
        data[i][index_title]=viehicle_power(m_full,high,width,r,Cd,eff_trans,g,lamata,motor_e,motor_z,i1,i0)
    except:
        pass

D:\bao.guanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in true_divide
D:\bao.guanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars


In [5]:
data.to_excel(r'D:\bao.guanyu\LocGit\2018222\vehicle_result.xlsx')

In [6]:
#18t 40km/h
m=16500
v=40
[a1,a2,a3]=841.5,7.05,0.165   #推荐阻力系数（m，v）
Fres=a1+a2*v/3.6+a3*v*v
P=Fres*v/3.6/1000
W100=P/v*100/0.8
print('40km/h等速能耗 %s kWh/100km'%round(W100,2))  #百公里能耗（效率80%）

40km/h等速能耗 41.11 kWh/100km
